In [1]:
import numpy as np
import tensorflow as tf
import time

In [17]:
N = 500 * 500

In [18]:
x = np.array(np.random.randn(N), dtype=np.float32)

## pytorch ops

In [19]:
import torch

In [20]:
Xt = torch.tensor(x).cuda()
torch.cuda.synchronize()

In [21]:
%%timeit
torch.cumsum(Xt, 0)
torch.cuda.synchronize()

30 µs ± 484 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [22]:
%%timeit
torch.sort(Xt, 0)
torch.cuda.synchronize()

980 µs ± 4.91 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Tensorflow ops

In [23]:
X = tf.placeholder(tf.float32, shape=(None,), name=None)
Y = tf.cumsum(X)
Z = tf.nn.top_k(X, tf.shape(X)[0])

config = tf.ConfigProto()
config.gpu_options.allow_growth=True

sess = tf.Session(config=config)

In [24]:
%%timeit
sess.run(Y, feed_dict={X: x})

63.8 ms ± 259 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
%%timeit
sess.run(Z, feed_dict={X: x})

2.8 ms ± 32.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
